In [39]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Q1: A window into NER
"""

from __future__ import absolute_import
from __future__ import division

import argparse
import sys
import time
import logging
from datetime import datetime

import tensorflow as tf

from util import print_sentence, write_conll
from data_util import load_and_preprocess_data, load_embeddings, read_conll, ModelHelper
from ner_model import NERModel
from defs import LBLS
#from report import Report

logger = logging.getLogger("hw3.q1")
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

In [40]:
class Config:
    """Holds model hyperparams and data information.

    The config class is used to store various hyperparameters and dataset
    information parameters. Model objects are passed a Config() object at
    instantiation.

    TODO: Fill in what n_window_features should be, using n_word_features and window_size.
    """
    n_word_features = 2 # Number of features for every word in the input.
    window_size = 1 # The size of the window to use.
    ### YOUR CODE HERE
    n_window_features = 6 # The total number of features used for each window.
    ### END YOUR CODE
    n_classes = 5
    dropout = 0.5
    embed_size = 50
    hidden_size = 200
    batch_size = 2048
    n_epochs = 10
    lr = 0.001

    def __init__(self, output_path=None):
        if output_path:
            # Where to save things.
            self.output_path = output_path
        else:
            self.output_path = "results/window/{:%Y%m%d_%H%M%S}/".format(datetime.now())
        self.model_output = self.output_path + "model.weights"
        self.eval_output = self.output_path + "results.txt"
        self.log_output = self.output_path + "log"
        self.conll_output = self.output_path + "window_predictions.conll"



In [41]:
def make_windowed_data(data, start, end, window_size = 1):
    """Uses the input sequences in @data to construct new windowed data points.

    TODO: In the code below, construct a window from each word in the
    input sentence by concatenating the words @window_size to the left
    and @window_size to the right to the word. Finally, add this new
    window data point and its label. to windowed_data.

    Args:
        data: is a list of (sentence, labels) tuples. @sentence is a list
            containing the words in the sentence and @label is a list of
            output labels. Each word is itself a list of
            @n_features features. For example, the sentence "Chris
            Manning is amazing" and labels "PER PER O O" would become
            ([[1,9], [2,9], [3,8], [4,8]], [1, 1, 4, 4]). Here "Chris"
            the word has been featurized as "[1, 9]", and "[1, 1, 4, 4]"
            is the list of labels.
        start: the featurized `start' token to be used for windows at the very
            beginning of the sentence.
        end: the featurized `end' token to be used for windows at the very
            end of the sentence.
        window_size: the length of the window to construct.
    Returns:
        a new list of data points, corresponding to each window in the
        sentence. Each data point consists of a list of
        @n_window_features features (corresponding to words from the
        window) to be used in the sentence and its NER label.
        If start=[5,8] and end=[6,8], the above example should return
        the list
        [([5, 8, 1, 9, 2, 9], 1),
         ([1, 9, 2, 9, 3, 8], 1),
         ...
         ]
    """

    windowed_data = []
    for sentence, labels in data:
		### YOUR CODE HERE (5-20 lines)
        sentence = window_size*[start]+sentence+window_size*[end]
        for i in range(window_size, len(sentence)-window_size):
            w_features = sentence[i-window_size:i+window_size+1]
            w_features = [y for x in w_features for y in x]
            label = labels[i-window_size]
            data_point = (w_features,label)       
            windowed_data.append(data_point)
		### END YOUR CODE
    print(windowed_data)
    return windowed_data


In [42]:
class WindowModel(NERModel):
    """
    Implements a feedforward neural network with an embedding layer and
    single hidden layer.
    This network will predict what label (e.g. PER) should be given to a
    given token (e.g. Manning) by  using a featurized window around the token.
    """

    def add_placeholders(self):
        """Generates placeholder variables to represent the input tensors

        These placeholders are used as inputs by the rest of the model building and will be fed
        data during training.  Note that when "None" is in a placeholder's shape, it's flexible
        (so we can use different batch sizes without rebuilding the model).

        Adds following nodes to the computational graph

        input_placeholder: Input placeholder tensor of  shape (None, n_window_features), type tf.int32
        labels_placeholder: Labels placeholder tensor of shape (None,), type tf.int32
        dropout_placeholder: Dropout value placeholder (scalar), type tf.float32

        Add these placeholders to self as the instance variables
            self.input_placeholder
            self.labels_placeholder
            self.dropout_placeholder

        (Don't change the variable names)
        """
        ### YOUR CODE HERE (~3-5 lines)
        self.input_placeholder = tf.placeholder(dtype=tf.int32, shape=(None, self.config.n_window_features))
        self.labels_placeholder = tf.placeholder(tf.int32,shape=(None,))
        self.dropout_placeholder = tf.placeholder(tf.float32)

        ### END YOUR CODE

    def create_feed_dict(self, inputs_batch, labels_batch=None, dropout=1):
        """Creates the feed_dict for the model.
        A feed_dict takes the form of:
        feed_dict = {
                <placeholder>: <tensor of values to be passed for placeholder>,
                ....
        }

        Hint: The keys for the feed_dict should be a subset of the placeholder
                    tensors created in add_placeholders.
        Hint: When an argument is None, don't add it to the feed_dict.

        Args:
            inputs_batch: A batch of input data.
            labels_batch: A batch of label data.
            dropout: The dropout rate.
        Returns:
            feed_dict: The feed dictionary mapping from placeholders to values.
        """
        ### YOUR CODE HERE (~5-10 lines)
        feed_dict = {self.input_placeholder: inputs_batch, self.dropout_placeholder:dropout}
        if labels_batch is not None:
            feed_dict[self.labels_placeholder] = labels_batch
        ### END YOUR CODE
        return feed_dict

    def add_embedding(self):
        """Adds an embedding layer that maps from input tokens (integers) to vectors and then
        concatenates those vectors:
            - Creates an embedding tensor and initializes it with self.pretrained_embeddings.
            - Uses the input_placeholder to index into the embeddings tensor, resulting in a
              tensor of shape (None, n_window_features, embedding_size).
            - Concatenates the embeddings by reshaping the embeddings tensor to shape
              (-1, n_window_features * embedding_size). Here -1 means variable length.

        Hint: You might find tf.nn.embedding_lookup useful.
        Hint: You can use tf.reshape to concatenate the vectors. See following link to understand
            what -1 in a shape means.
            https://www.tensorflow.org/api_docs/python/array_ops/shapes_and_shaping#reshape.
        Returns:
            embeddings: tf.Tensor of shape (None, n_window_features*embed_size)
        """
        ### YOUR CODE HERE (!3-5 lines)
        embeddings = tf.Variable(self.pretrained_embeddings)
        embeddings = tf.nn.embedding_lookup(embeddings, self.input_placeholder)    
        embeddings = tf.reshape(embeddings, (-1, self.config.n_window_features * self.config.embed_size))
                                  
                                                                                                                 
        ### END YOUR CODE
        return embeddings

    def add_prediction_op(self):
        """Adds the 1-hidden-layer NN:
            h = Relu(xW + b1)
            h_drop = Dropout(h, dropout_rate)
            pred = h_dropU + b2

        Recall that we are not applying a softmax to pred. The softmax will instead be done in
        the add_loss_op function, which improves efficiency because we can use
        tf.nn.softmax_cross_entropy_with_logits

        When creating a new variable, use the tf.get_variable function
        because it lets us specify an initializer.

        Use tf.contrib.layers.xavier_initializer to initialize matrices.
        This is TensorFlow's implementation of the Xavier initialization
        trick we used in last assignment.

        Note: tf.nn.dropout takes the keep probability (1 - p_drop) as an argument.
            The keep probability should be set to the value of dropout_rate.

        Returns:
            pred: tf.Tensor of shape (batch_size, n_classes)
        """

        x = self.add_embedding()
        dropout_rate = self.dropout_placeholder
        ### YOUR CODE HERE (~10-20 lines)
        W = tf.get_variable("W",dtype=tf.float32, shape=(self.config.n_window_features*self.config.embed_size,self.config.hidden_size), initializer=tf.contrib.layers.xavier_initializer())
        b1 = tf.Variable(tf.zeros((self.config.hidden_size,)))
        U = tf.get_variable("U",dtype=tf.float32, shape=(self.config.hidden_size,self.config.n_classes), initializer=tf.contrib.layers.xavier_initializer())
        b2 = tf.Variable(tf.zeros((self.config.n_classes,)))
        h = tf.nn.relu(tf.matmul(x,W)+b1)
        h_drop = tf.nn.dropout(h,dropout_rate)
        pred = tf.matmul(h_drop,U)+b2
        ### END YOUR CODE
        return pred

    def add_loss_op(self, pred):
        """Adds Ops for the loss function to the computational graph.
        In this case we are using cross entropy loss.
        The loss should be averaged over all examples in the current minibatch.

        Remember that you can use tf.nn.sparse_softmax_cross_entropy_with_logits to simplify your
        implementation. You might find tf.reduce_mean useful.
        Args:
            pred: A tensor of shape (batch_size, n_classes) containing the output of the neural
                  network before the softmax layer.
        Returns:
            loss: A 0-d tensor (scalar)
        """
        ### YOUR CODE HERE (~2-5 lines)
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.labels_placeholder, logits=pred))                        
        ### END YOUR CODE
        return loss

    def add_training_op(self, loss):
        """Sets up the training Ops.

        Creates an optimizer and applies the gradients to all trainable variables.
        The Op returned by this function is what must be passed to the
        `sess.run()` call to cause the model to train. See

        https://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#Optimizer

        for more information.

        Use tf.train.AdamOptimizer for this model.
        Calling optimizer.minimize() will return a train_op object.

        Args:
            loss: Loss tensor, from cross_entropy_loss.
        Returns:
            train_op: The Op for training.
        """
        ### YOUR CODE HERE (~1-2 lines)
        train_op = tf.train.AdamOptimizer(self.config.lr).minimize(loss)

        ### END YOUR CODE
        return train_op

    def preprocess_sequence_data(self, examples):
        return make_windowed_data(examples, start=self.helper.START, end=self.helper.END, window_size=self.config.window_size)

    def consolidate_predictions(self, examples_raw, examples, preds):
        """Batch the predictions into groups of sentence length.
        """
        ret = []
        #pdb.set_trace()
        i = 0
        for sentence, labels in examples_raw:
            labels_ = preds[i:i+len(sentence)]
            i += len(sentence)
            ret.append([sentence, labels, labels_])
        return ret

    def predict_on_batch(self, sess, inputs_batch):
        """Make predictions for the provided batch of data

        Args:
            sess: tf.Session()
            input_batch: np.ndarray of shape (n_samples, n_features)
        Returns:
            predictions: np.ndarray of shape (n_samples, n_classes)
        """
        feed = self.create_feed_dict(inputs_batch)
        predictions = sess.run(tf.argmax(self.pred, axis=1), feed_dict=feed)
        return predictions

    def train_on_batch(self, sess, inputs_batch, labels_batch):
        feed = self.create_feed_dict(inputs_batch, labels_batch=labels_batch,
                                     dropout=self.config.dropout)
        _, loss = sess.run([self.train_op, self.loss], feed_dict=feed)
        return loss

    def __init__(self, helper, config, pretrained_embeddings, report=None):
        super(WindowModel, self).__init__(helper, config, report)
        self.pretrained_embeddings = pretrained_embeddings

        # Defining placeholders.
        self.input_placeholder = None
        self.labels_placeholder = None
        self.dropout_placeholder = None

        self.build()


SyntaxError: invalid syntax (<ipython-input-42-5c8b460e7b94>, line 116)

In [43]:
def test_make_windowed_data():
    sentences = [[[1,1], [2,0], [3,3]]]
    sentence_labels = [[1, 2, 3]]
    data = zip(sentences, sentence_labels)
    w_data = make_windowed_data(data, start=[5,0], end=[6,0], window_size=1)

    assert len(w_data) == sum(len(sentence) for sentence in sentences)

    assert w_data == [
        ([5,0] + [1,1] + [2,0], 1,),
        ([1,1] + [2,0] + [3,3], 2,),
        ([2,0] + [3,3] + [6,0], 3,),
        ]

def do_test1(_):
    logger.info("Testing make_windowed_data")
    test_make_windowed_data()
    logger.info("Passed!")

def do_test2(args):
    logger.info("Testing implementation of WindowModel")
    config = Config()
    helper, train, dev, train_raw, dev_raw = load_and_preprocess_data(args)
    embeddings = load_embeddings(args, helper)
    config.embed_size = embeddings.shape[1]

    with tf.Graph().as_default():
        logger.info("Building model...",)
        start = time.time()
        model = WindowModel(helper, config, embeddings)
        logger.info("took %.2f seconds", time.time() - start)

        init = tf.global_variables_initializer()
        saver = None

        with tf.Session() as session:
            session.run(init)
            model.fit(session, saver, train, dev)

    logger.info("Model did not crash!")
    logger.info("Passed!")

def do_train(args):
    # Set up some parameters.
    config = Config()
    helper, train, dev, train_raw, dev_raw = load_and_preprocess_data(args)
    embeddings = load_embeddings(args, helper)
    config.embed_size = embeddings.shape[1]
    helper.save(config.output_path)

    handler = logging.FileHandler(config.log_output)
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
    logging.getLogger().addHandler(handler)

    report = None #Report(Config.eval_output)

    with tf.Graph().as_default():
        logger.info("Building model...",)
        start = time.time()
        model = WindowModel(helper, config, embeddings)
        logger.info("took %.2f seconds", time.time() - start)

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        with tf.Session() as session:
            session.run(init)
            model.fit(session, saver, train, dev)
            if report:
                report.log_output(model.output(session, dev_raw))
                report.save()
            else:
                # Save predictions in a text file.
                output = model.output(session, dev_raw)
                sentences, labels, predictions = zip(*output)
                predictions = [[LBLS[l] for l in preds] for preds in predictions]
                output = zip(sentences, labels, predictions)

                with open(model.config.conll_output, 'w') as f:
                    write_conll(f, output)
                with open(model.config.eval_output, 'w') as f:
                    for sentence, labels, predictions in output:
                        print_sentence(f, sentence, labels, predictions)

def do_evaluate(args):
    config = Config(args.model_path)
    helper = ModelHelper.load(args.model_path)
    input_data = read_conll(args.data)
    embeddings = load_embeddings(args, helper)
    config.embed_size = embeddings.shape[1]

    with tf.Graph().as_default():
        logger.info("Building model...",)
        start = time.time()
        model = WindowModel(helper, config, embeddings)

        logger.info("took %.2f seconds", time.time() - start)

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        with tf.Session() as session:
            session.run(init)
            saver.restore(session, model.config.model_output)
            for sentence, labels, predictions in model.output(session, input_data):
                predictions = [LBLS[l] for l in predictions]
                print_sentence(args.output, sentence, labels, predictions)

def do_shell(args):
    config = Config(args.model_path)
    helper = ModelHelper.load(args.model_path)
    embeddings = load_embeddings(args, helper)
    config.embed_size = embeddings.shape[1]

    with tf.Graph().as_default():
        logger.info("Building model...",)
        start = time.time()
        model = WindowModel(helper, config, embeddings)
        logger.info("took %.2f seconds", time.time() - start)

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        with tf.Session() as session:
            session.run(init)
            saver.restore(session, model.config.model_output)

            print("""Welcome!
You can use this shell to explore the behavior of your model.
Please enter sentences with spaces between tokens, e.g.,
input> Germany 's representative to the European Union 's veterinary committee .
""")
            while True:
                # Create simple REPL
                try:
                    sentence = raw_input("input> ")
                    tokens = sentence.strip().split(" ")
                    for sentence, _, predictions in model.output(session, [(tokens, ["O"] * len(tokens))]):
                        predictions = [LBLS[l] for l in predictions]
                        print_sentence(sys.stdout, sentence, [""] * len(tokens), predictions)
                except EOFError:
                    print("Closing session.")
                    break

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Trains and tests an NER model')
    subparsers = parser.add_subparsers()

    command_parser = subparsers.add_parser('test1', help='')
    command_parser.set_defaults(func=do_test1)

    command_parser = subparsers.add_parser('test2', help='')
    command_parser.add_argument('-dt', '--data-train', type=argparse.FileType('r'), default="data/tiny.conll", help="Training data")
    command_parser.add_argument('-dd', '--data-dev', type=argparse.FileType('r'), default="data/tiny.conll", help="Dev data")
    command_parser.add_argument('-v', '--vocab', type=argparse.FileType('r'), default="data/vocab.txt", help="Path to vocabulary file")
    command_parser.add_argument('-vv', '--vectors', type=argparse.FileType('r'), default="data/wordVectors.txt", help="Path to word vectors file")
    command_parser.set_defaults(func=do_test2)

    command_parser = subparsers.add_parser('train', help='')
    command_parser.add_argument('-dt', '--data-train', type=argparse.FileType('r'), default="data/train.conll", help="Training data")
    command_parser.add_argument('-dd', '--data-dev', type=argparse.FileType('r'), default="data/dev.conll", help="Dev data")
    command_parser.add_argument('-v', '--vocab', type=argparse.FileType('r'), default="data/vocab.txt", help="Path to vocabulary file")
    command_parser.add_argument('-vv', '--vectors', type=argparse.FileType('r'), default="data/wordVectors.txt", help="Path to word vectors file")
    command_parser.set_defaults(func=do_train)

    command_parser = subparsers.add_parser('evaluate', help='')
    command_parser.add_argument('-d', '--data', type=argparse.FileType('r'), default="data/dev.conll", help="Training data")
    command_parser.add_argument('-m', '--model-path', help="Training data")
    command_parser.add_argument('-v', '--vocab', type=argparse.FileType('r'), default="data/vocab.txt", help="Path to vocabulary file")
    command_parser.add_argument('-vv', '--vectors', type=argparse.FileType('r'), default="data/wordVectors.txt", help="Path to word vectors file")
    command_parser.add_argument('-o', '--output', type=argparse.FileType('w'), default=sys.stdout, help="Training data")
    command_parser.set_defaults(func=do_evaluate)

    command_parser = subparsers.add_parser('shell', help='')
    command_parser.add_argument('-m', '--model-path', help="Training data")
    command_parser.add_argument('-v', '--vocab', type=argparse.FileType('r'), default="data/vocab.txt", help="Path to vocabulary file")
    command_parser.add_argument('-vv', '--vectors', type=argparse.FileType('r'), default="data/wordVectors.txt", help="Path to word vectors file")
    command_parser.set_defaults(func=do_shell)

    ARGS = parser.parse_args()
    if ARGS.func is None:
        parser.print_help()
        sys.exit(1)
    else:
        ARGS.func(ARGS)


usage: ipykernel_launcher.py [-h] {test1,test2,train,evaluate,shell} ...
ipykernel_launcher.py: error: invalid choice: '/Users/lhd0430/Library/Jupyter/runtime/kernel-99cbc93b-5c4c-4dac-957a-b3f79aa092f4.json' (choose from 'test1', 'test2', 'train', 'evaluate', 'shell')


SystemExit: 2

/Users/lhd0430/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [44]:
test_make_windowed_data()

[([5, 0, 1, 1, 2, 0], 1), ([1, 1, 2, 0, 3, 3], 2), ([2, 0, 3, 3, 6, 0], 3)]


In [45]:
do_test1(_)

INFO:Testing make_windowed_data
INFO:Passed!


[([5, 0, 1, 1, 2, 0], 1), ([1, 1, 2, 0, 3, 3], 2), ([2, 0, 3, 3, 6, 0], 3)]


In [50]:
do_test2('test2')

INFO:Testing implementation of WindowModel
INFO:Loading training data...


AttributeError: 'str' object has no attribute 'data_train'

In [63]:
!python q1_window.py test2

/Users/lhd0430/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
INFO:Testing implementation of WindowModel
INFO:Loading training data...
INFO:Done. Read 721 sentences
INFO:Loading dev data...
INFO:Done. Read 721 sentences
INFO:Built dictionary for 2653 features.
INFO:Initialized embeddings.
INFO:Building model...
INFO:took 5.55 seconds


[([2650, 2646, 89, 2647, 1070, 2646], 1), ([89, 2647, 1070, 2646, 115, 2648], 4), ([1070, 2646, 115, 2648, 288, 2646], 3), ([115, 2648, 288, 2646, 7, 2646], 4), ([288, 2646, 7, 2646, 1071, 2646], 4), ([7, 2646, 1071, 2646, 78, 2648], 4), ([1071, 2646, 78, 2648, 392, 2646], 3), ([78, 2648, 392, 2646, 1, 2649], 4), ([392, 2646, 1, 2649, 2651, 2646], 4), ([2650, 2646, 1072, 2648, 175, 2648], 0), ([1072, 2648, 175, 2648, 2651, 2646], 0), ([2650, 2646, 1073, 2647, 20, 2649], 2), ([1073, 2647, 20, 2649, 2651, 2646], 4), ([2650, 2646, 4, 2648, 116, 2648], 4), ([4, 2648, 116, 2648, 393, 2648], 1), ([116, 2648, 393, 2648, 17, 2646], 1), ([393, 2648, 17, 2646, 11, 2646], 4), ([17, 2646, 11, 2646, 22, 2648], 4), ([11, 2646, 22, 2648, 29, 2646], 4), ([22, 2648, 29, 2646, 1074, 2646], 4), ([29, 2646, 1074, 2646, 25, 2646], 4), ([1074, 2646, 25, 2646, 115, 2648], 4), ([25, 2646, 115, 2648, 394, 2646], 3), ([115, 2648, 394, 2646, 7, 2646], 4), ([394, 2646, 7, 2646, 133, 2646], 4), ([7, 2646, 133, 264

 4), ([2650, 2646, 2645, 2648, 2646, 2648], 0), ([2645, 2648, 2646, 2648, 2, 2649], 0), ([2646, 2648, 2, 2649, 278, 2648], 4), ([2, 2649, 278, 2648, 3, 2649], 2), ([278, 2648, 3, 2649, 24, 2646], 4), ([3, 2649, 24, 2646, 6, 2649], 4), ([24, 2646, 6, 2649, 10, 2649], 4), ([6, 2649, 10, 2649, 982, 2648], 4), ([10, 2649, 982, 2648, 247, 2648], 0), ([982, 2648, 247, 2648, 2, 2649], 0), ([247, 2648, 2, 2649, 36, 2647], 4), ([2, 2649, 36, 2647, 3, 2649], 2), ([36, 2647, 3, 2649, 2651, 2646], 4)]


[([2650, 2646, 89, 2647, 1070, 2646], 1), ([89, 2647, 1070, 2646, 115, 2648], 4), ([1070, 2646, 115, 2648, 288, 2646], 3), ([115, 2648, 288, 2646, 7, 2646], 4), ([288, 2646, 7, 2646, 1071, 2646], 4), ([7, 2646, 1071, 2646, 78, 2648], 4), ([1071, 2646, 78, 2648, 392, 2646], 3), ([78, 2648, 392, 2646, 1, 2649], 4), ([392, 2646, 1, 2649, 2651, 2646], 4), ([2650, 2646, 1072, 2648, 175, 2648], 0), ([1072, 2648, 175, 2648, 2651, 2646], 0), ([2650, 2646, 1073, 2647, 20, 2649], 2), ([1073, 2647, 20, 2649, 2651, 2646], 4), ([2650, 2646, 4, 2648, 116, 2648], 4), ([4, 2648, 116, 2648, 393, 2648], 1), ([116, 2648, 393, 2648, 17, 2646], 1), ([393, 2648, 17, 2646, 11, 2646], 4), ([17, 2646, 11, 2646, 22, 2648], 4), ([11, 2646, 22, 2648, 29, 2646], 4), ([22, 2648, 29, 2646, 1074, 2646], 4), ([29, 2646, 1074, 2646, 25, 2646], 4), ([1074, 2646, 25, 2646, 115, 2648], 4), ([25, 2646, 115, 2648, 394, 2646], 3), ([115, 2648, 394, 2646, 7, 2646], 4), ([394, 2646, 7, 2646, 133, 2646], 4), ([7, 2646, 133, 264

46, 2648, 2496, 2648], 2), ([146, 2648, 2496, 2648, 3, 2649], 2), ([2496, 2648, 3, 2649, 2497, 2649], 4), ([3, 2649, 2497, 2649, 2651, 2646], 4), ([2650, 2646, 1063, 2649, 2651, 2646], 4), ([2650, 2646, 375, 2648, 2498, 2648], 0), ([375, 2648, 2498, 2648, 2, 2649], 0), ([2498, 2648, 2, 2649, 216, 2648], 4), ([2, 2649, 216, 2648, 3, 2649], 2), ([216, 2648, 3, 2649, 23, 2646], 4), ([3, 2649, 23, 2646, 2499, 2648], 4), ([23, 2646, 2499, 2648, 2500, 2648], 0), ([2499, 2648, 2500, 2648, 2501, 2648], 0), ([2500, 2648, 2501, 2648, 2, 2649], 0), ([2501, 2648, 2, 2649, 216, 2648], 4), ([2, 2649, 216, 2648, 3, 2649], 2), ([216, 2648, 3, 2649, 2502, 2649], 4), ([3, 2649, 2502, 2649, 2651, 2646], 4), ([2650, 2646, 2503, 2649, 2504, 2649], 4), ([2503, 2649, 2504, 2649, 2651, 2646], 4), ([2650, 2646, 546, 2647, 2505, 2648], 0), ([546, 2647, 2505, 2648, 2, 2649], 0), ([2505, 2648, 2, 2649, 216, 2648], 4), ([2, 2649, 216, 2648, 3, 2649], 2), ([216, 2648, 3, 2649, 23, 2646], 4), ([3, 2649, 23, 2646, 10

5/5 [==============================] - 0s     
DEBUG:Token-level confusion matrix:
go\gu  	PER    	ORG    	LOC    	MISC   	O      
PER    	422.00 	40.00  	341.00 	2.00   	19.00  
ORG    	150.00 	51.00  	189.00 	0.00   	1.00   
LOC    	285.00 	29.00  	262.00 	0.00   	1.00   
MISC   	81.00  	19.00  	158.00 	0.00   	0.00   
O      	3484.00	1043.00	2545.00	5.00   	39.00  

DEBUG:Token-level scores:
label	acc  	prec 	rec  	f1   
PER  	0.52 	0.10 	0.51 	0.16 
ORG  	0.84 	0.04 	0.13 	0.06 
LOC  	0.61 	0.07 	0.45 	0.13 
MISC 	0.00 	0.00 	0.00 	0.00 
O    	0.23 	0.65 	0.01 	0.01 
micro	0.63 	0.08 	0.08 	0.08 
macro	0.44 	0.17 	0.22 	0.07 
not-O	0.74 	0.08 	0.36 	0.13 

INFO:Entity level P/R/F1: 0.02/0.09/0.04

INFO:Epoch 2 out of 10
INFO:Evaluating on development data
5/5 [==============================] - 0s     
DEBUG:Token-level confusion matrix:
go\gu  	PER    	ORG    	LOC    	MISC   	O      
PER    	422.00 	40.00  	341.00 	2.00   	19.00  
ORG    	150.00 	51.00  	189.00 	0.00   	1.00   
LOC